In [1]:
print('hello world')

In [2]:
from strategy1 import Gold_5min_Strategy
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
from backtesting.test import SMA
import yfinance as yf
import pandas as pd

In [3]:
gold_data=pd.read_csv('XAU_5m_data_2004_to_2024-09-20.csv')
print(gold_data.head())

In [4]:
gold_data['Datetime'] = pd.to_datetime(gold_data['Date']+' '+gold_data['Time'])
print(gold_data.head())

Reseting the index for the dataset and deleting unwanted columns



In [5]:
gold_data.drop(columns=['Date', 'Time'], inplace=True)
gold_data.dropna(inplace=True)
gold_data.set_index('Datetime', inplace=True)
print(gold_data.head())


In [8]:
gold_data.to_csv('gold_5m_data2.csv')

In [17]:

print(gold_data.info())
print(gold_data.tail(100))

In [15]:
gold_data2=pd.read_csv('XAU_5m_data_2004_to_2024-09-20.csv')
print(gold_data2.info())

In [ ]:
print('gold_data2.head()')